<a href="https://colab.research.google.com/github/OlegLaiok/Comp_Lingv/blob/homework9/%D0%9B%D0%B0%D0%B19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install natasha

     |████████████████████████████████| 34.4MB 117kB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 8.2MB 15.1MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26120 sha256=1dce57e313945e53f465c1e4ddb57031892028996a774fbad5286c2e85dcfb18
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [3]:
import natasha
import json
import re
import pandas as pd
from natasha import Doc, Segmenter, NewsNERTagger, NewsEmbedding, NewsMorphTagger, MorphVocab
import pymorphy2 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans
analyzer = pymorphy2.MorphAnalyzer()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open("/content/drive/MyDrive/Colab Notebooks/VK_data_lentach_cleared.json", "r", encoding ='utf-8') as f:
    text = json.load(f)

Для выделения имменованных сущностей будем использовать тот же корпус новостей.

In [6]:
embedding = NewsEmbedding() 
segmenter = Segmenter() # сегментация текста
morph_tagger = NewsMorphTagger(embedding) #морфологический анализ для дальейшей лемматизации и выделения тэгов именованных сущностей
morph_vocab = MorphVocab() 
ner_tagger = NewsNERTagger(embedding) #тэггер имменованных сущностей
df = pd.DataFrame.from_dict({'text':text}) #создадим датафрейм, где будем хранить исходный текст, именнованные сущности и номера кластеров

In [7]:
def get_ner(transcript):
  script = Doc(re.sub(r'\((.*?)\)', "", transcript))
  script.segment(segmenter)
  script.tag_morph(morph_tagger)
  for token in script.tokens:
    token.lemmatize(morph_vocab)
  script.tag_ner(ner_tagger)
  for span in script.spans:
    span.normalize(morph_vocab)
  named_ents = [(i.text, i.type, i.normal) for i in script.spans]
  normed_ents = []
  for word, tag, norm in named_ents:
    if len(word.split()) == 1 and tag == "LOC":
      for gram in range(len(analyzer.parse(word))):
        if "Geox" in analyzer.parse(word)[gram].tag:
          normed_ents.append((analyzer.parse(word)[gram].normal_form))
          break
        elif gram == len(analyzer.parse(word)) - 1:
          normed_ents.append((norm.lower().strip(".,!?;-")))
    else:
      normed_ents.append((norm.lower().strip(".,!?;-")))
  return sorted(normed_ents)

Следующая функция работает таким образом:
1. текст сегментируется на токены
2. на токены накладывается частеречная разметка
4. токены лемматизируются
5. выделяются именованные сущности
6. токены нормализуются
7. обрабатывается нормализация географических объектов, т.к. в процессе выяснилось, что natasha хуже их нормализует, чем pymorphy2.



In [8]:
normed_ent = []
for item in text:
    normed_ent.append(get_ner(item)) # из текстовых данных выделяем именнованные сущность для каждого документа и записываем их в датафрейм
df['named_entities'] = normed_ent
has_ner = [i for i in df.index.values if df.named_entities[i]] #составляем список не пустых именнованных сущностей
print(len(has_ner), df.shape[0]) # количество записей без пустых сущностей и с пустыми
df_ner = df[df.index.isin(has_ner)] #создаем датафрейм где у каждого текстового документа обязательно есть не пустые имен. сущности
ner_voc = []
for row in df_ner.named_entities.tolist():
  ner_voc.extend(row) 
len(ner_voc), len(set(ner_voc)) #количество всех сущностей в документе и количество уникальных сущностей

780 894


(2855, 1151)

In [10]:
vocab = sorted(set(ner_voc)) #создаем словарь сущностей
corpus = df_ner.named_entities.apply(str).tolist() #создаем корпус всех сущностей
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocab)),
                 ('tfid', TfidfTransformer())]).fit(corpus) #создаем конвеер преобразований
X = pipe.fit_transform(corpus) #применяем конвеер к корпусу

In [11]:
km = KMeans(n_clusters=20, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True) #обучаем класторизацию на корпусе
km.fit(X) #применяем класторизацию к корпусу
print('Силуэт: ', metrics.silhouette_score(X, km.labels_, sample_size=1000))
print('Индекс Дэвиса-Болдуина: ', metrics.davies_bouldin_score(X.toarray(), km.labels_))

Силуэт:  0.13952876651282592
Индекс Дэвиса-Болдуина:  1.9166579849583538


Для нахождения оптимального количесва кластеров будем ориентироваться на метрики Силует и Индекс Девиса-Болдуина

In [12]:
km = KMeans(n_clusters=25, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)
km.fit(X)
print('Силуэт: ', metrics.silhouette_score(X, km.labels_, sample_size=1000))
print('Индекс Дэвиса-Болдуина: ', metrics.davies_bouldin_score(X.toarray(), km.labels_))

Силуэт:  0.1484027072614288
Индекс Дэвиса-Болдуина:  1.6406909399449683


In [13]:
km = KMeans(n_clusters=30, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)
km.fit(X)
print('Силуэт: ', metrics.silhouette_score(X, km.labels_, sample_size=1000))
print('Индекс Дэвиса-Болдуина: ', metrics.davies_bouldin_score(X.toarray(), km.labels_))

Силуэт:  0.16373586363757117
Индекс Дэвиса-Болдуина:  1.737259414689813


In [14]:
km = KMeans(n_clusters=35, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)
km.fit(X)
print('Силуэт: ', metrics.silhouette_score(X, km.labels_, sample_size=1000))
print('Индекс Дэвиса-Болдуина: ', metrics.davies_bouldin_score(X.toarray(), km.labels_))

Силуэт:  0.17759318049812903
Индекс Дэвиса-Болдуина:  1.6794265900769187


Остановимся на 40 кластерах, т.к. дальше Силуэт очень медленно растет, а Индекс Девиса-Болдуина наоборот сильно начинает расти.

In [16]:
df_ner["label"] = km.predict(X)
print(df_ner["label"].value_counts()) #посмотрим сколько объектов пинадлежит каждому классу

1     459
3      45
18     27
12     25
11     23
24     21
5      16
16     16
25     13
31     12
8      12
33     11
34      9
13      7
14      7
17      7
26      7
15      6
27      6
6       6
30      5
20      5
28      5
0       4
21      4
7       3
29      3
19      3
10      3
22      3
2       2
9       2
23      1
4       1
32      1
Name: label, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Заметим, что большое количесво объектов сгрупперовалось в первом кластере, скорее всего там находятся данные, которые алгоритм не смог определить в другие классы. Посмотрим примеры сущностей, содержащихся в этих статьях

In [17]:
df_ner.query("label == 1").sample(10)

,text,named_entities,label
655,Объявлены номинанты на «Оскар-2021»: Лучший фи...,"[андра дэй, ванесса кирби, виола дэвис, гэри о...",1
395,"Måneskin — итальянская рок-группа, которая буд...","[måneskin, италия]",1
736,Министра здравоохранения Республики Алтай заде...,[республика алтай],1
94,Вечерняя сводка новостей по коронавирусу: Инди...,"[regeneron pharmaceuticals, воз, зузана чапуто...",1
10,Кошка села на шею бабушке. В Ярославле появил...,[ярославль],1
765,В РЖД планируют привести все железнодорожные в...,[ржд],1
846,Журнал Time поместил на обложку основательниц...,"[time, анна ривину, калининград, лувр, насилию...",1
306,В Благовещенске бригада врачей провела операци...,"[благовещенск, благовещенск]",1
83,«Ведомости»: аптеки не хотят заниматься продаж...,"[ведомости, минздрав]",1
63,Хочешь рассмешить бога — расскажи ему о своих ...,"[scmp, аргентина, китай, ковивак, минздрав, но...",1


Видим, что тут содержатся документы имеющие нетипичные темы, поэтому кластеризация для них прошла не совсем корректно.
Выведем получившиеся кластеры имменованных сущностей:

In [18]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(35):
  print("Cluster %d:" % i, end='')
  for ind in order_centroids[i, :10]:
    print(' %s' % terms[ind], end='')
  print()

Cluster 0: европа еврокомиссия турция россия гонконг токио гамалея франция япония сми
Cluster 1: россия astrazeneca турция воз германия сша франция ск беларусь ес
Cluster 2: великобритания ярославль джон магуфули джастин трюдо джеймс кэмерон джек дорси джефф безос джимми хендрикс джо байден джордж флойд
Cluster 3: петербург москва россия ярославль джастин трюдо джеймс кэмерон джек дорси джефф безос джимми хендрикс джо байден
Cluster 4: aliexpress россия джон магуфули джастин трюдо джеймс кэмерон джек дорси джефф безос джимми хендрикс джо байден ярославль
Cluster 5: москва россия маяковский каро благовещенск омон андрей мтс петербург казань
Cluster 6: роскомнадзор восточный сми байден сша россия детская студия «дом свет джастин трюдо джеймс кэмерон джек дорси
Cluster 7: it лужниках facebook дагестан госдума сми путин россия джимми хендрикс детская студия «дом свет
Cluster 8: лентач голосовалка lumen кровосток dj пикчерами аукцыон главклуб clubhouse питер
Cluster 9: париж ярославль джон 

Не смотря на то, что кластеризация получилась не идеальная, в каждом кластере прослеживаются какие-то ключевые темы. Например какие-то кластеры будут содержать сущности из новостей про криминал (кластер 34), внешнюю политику(кластер 15), IT (кластер 33) и тд.